# Hugging Face and Sagemaker: fine-tuning Pegasus with MEDLINE/PubMed data


# Introduction

In this demo, we will use the [Hugging Face transformers](https://huggingface.co/docs/transformers/index) library to fine-tune the Pegasus model on the Medline PubMed dataset for text summarization tasks. We will then evaluate the performance of the resulting model using various metrics and techniques. Finally, we will deploy the model for inference on a [SageMaker](https://aws.amazon.com/sagemaker/) Endpoint, allowing us to generate text summaries quickly and efficiently. 


## The Model

[Pegasus](https://huggingface.co/docs/transformers/model_doc/pegasus) is a transformer-based model that was introduced by Google AI in 2020. It is specifically designed for abstractive text summarization tasks and has shown impressive results in various benchmark datasets.

## The Data

The [Medline PubMed](https://huggingface.co/datasets/scientific_papers/viewer/pubmed/train) dataset is a widely-used collection of scientific research articles in the field of biomedical sciences. It contains millions of abstracts and citations from various research journals and publications.

## Setup 

[Pegasus](https://huggingface.co/docs/transformers/model_doc/pegasus) is a transformer-based model that was introduced by Google AI in 2020. It is specifically designed for abstractive text summarization tasks and has shown impressive results in various benchmark datasets.

## Dependencies

First, you need to install the required dependencies

In [ ]:
%pip install transformers --quiet
%pip install nltk --quiet
%pip install accelerate --quiet
%pip install datasets --quiet
%pip install rouge_score --quiet
%pip install evaluate --quiet

### Variables and hyperparameters

In [ ]:
from datetime import datetime

# vars
model_checkpoint = 'google/pegasus-xsum'
bucket_name = 'pegsum-content-bucket'
artifact_path = 'training_artifacts/%s/' % datetime.today().strftime('%Y-%m-%d') 

# tokenizer
max_target_length = 32
max_input_length = 512
ds_col_full = "article"
ds_col_summ = "abstract"

# training
batch_size = 1
num_train_epochs = 5
learning_rate = .001
optimizer_name = 'Adam' # must be a supported algorithm from https://pytorch.org/docs/stable/optim.html

In [ ]:
import boto3
s3 = boto3.client('s3')

In [ ]:
from datasets import load_dataset
dataset = load_dataset("scientific_papers", "pubmed")

### Tokenizer
Prepares data for the model by mapping text into numerical inputs called tokens

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples[ds_col_full],
        max_length=max_input_length,
        truncation=True,
        padding='max_length'
    )
    labels = tokenizer(
        examples[ds_col_summ], max_length=max_target_length, truncation=True, padding='max_length'
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

tokenized_datasets.set_format("torch")

tokenized_datasets = tokenized_datasets.remove_columns(
    dataset["train"].column_names
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

### Data Collator
Pads data during batching

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

### Data Loader
Incrementally loads data from the dataset

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=batch_size
)

### Optimizer
The optimizer maintains training state and update parameters based on training loss

In [ ]:
### Hardcode the optimizer, replaced by following code block

#from torch.optim import Adam

#optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Dynamically select optimizer based on input var

from importlib import import_module

module = import_module('torch.optim')
opt_fnc = getattr(module, optimizer_name)

optimizer = opt_fnc(model.parameters(), lr=learning_rate)

### Accelerator
The accelerator enables distributed training

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

### Learning rate scheduler
Manages adjustments to the learning rate

In [ ]:
from transformers import get_scheduler

num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")

In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np
import nltk

nltk.download('punkt')
progress_bar = tqdm(range(num_training_steps))
epoch_scores = []

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    result = {key: value * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    
    # Save epoch score
    epoch_score = "Epoch %s: %s" % (epoch, result)
    epoch_scores.append(epoch_score)
    print(epoch_score)

    # Save model for this epoch
    epoch_name = "epoch_%s/" % epoch  
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained('model_dir', save_function=accelerator.save)
    
    # Upload epoch artifacts to S3
    with open("model_dir/pytorch_model.bin", "rb") as f:
        s3.upload_fileobj(f, bucket_name, artifact_path + epoch_name + "pytorch_model.bin")
    with open("model_dir/generation_config.json", "rb") as f:
        s3.upload_fileobj(f, bucket_name, artifact_path + epoch_name + "generation_config.json")
    with open("model_dir/config.json", "rb") as f:
        s3.upload_fileobj(f, bucket_name, artifact_path + epoch_name + "config.json")

### Write each epoch's rouge scores to file

In [ ]:
with open("epoch_scores.txt", "w") as f:
    for entry in epoch_scores:
        f.write(entry + "\n")

### Save scores and tokenizer to S3

In [ ]:
with open("epoch_scores.txt", "rb") as f:
    s3.upload_fileobj(f, bucket_name, artifact_path + "epoch_scores.txt")

In [ ]:
tokenizer.save_pretrained('model_dir')

In [ ]:
with open("model_dir/special_tokens_map.json", "rb") as f:
    s3.upload_fileobj(f, bucket_name, artifact_path + "special_tokens_map.json")
with open("model_dir/tokenizer_config.json", "rb") as f:
    s3.upload_fileobj(f, bucket_name, artifact_path + "tokenizer_config.json")
with open("model_dir/tokenizer.json", "rb") as f:
    s3.upload_fileobj(f, bucket_name, artifact_path + "tokenizer.json")

### Zip and save the model to S3

In [ ]:
cd model_dir/
!tar -czvf model.tar.gz *

In [ ]:
with open("model.tar.gz", "rb") as f:
    s3.upload_fileobj(f, bucket_name, artifact_path + "model/model.tar.gz")